## STRAICO

In [ ]:
import json
import csv
import os
import requests
from typing import List, Optional, Dict, Any
from tqdm import tqdm

# API endpoint
API_URL = "https://api.straico.com/v1/prompt/completion"

# Hardcoded API key and default model(s)
API_KEY = "v8-Ps43kVOZynO8LHli5ay9oXNDR7VNMmDkxONui5xcp8Q0tjAo"

PROMPT_ID_MAP = {
    "prompt_1": "P1", # Dada la palabra --> Definición del modismo.
    "prompt_2": "P2", # Dada la palabra --> Decir si es modismo o no.
    "prompt_3": "P3", # Dado el ejemplo --> Reemplazo literal y definición del remplazo.
}

#### Usar Modelos

In [ ]:
def send_prompt(message: str, models: Optional[List[str]] = None) -> Dict[str, Any]:
    
    """ Send a single text prompt to Straico and return the parsed response.
        This function always performs a live HTTP request.
    """

    payload: Dict[str, Any] = {"models": models, "message": message}
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    try:
        resp = requests.post(API_URL, headers=headers, json=payload, timeout=30)
    except requests.exceptions.RequestException as exc:
        return {"error": str(exc)}

    # Parse JSON if available
    try:
        data = resp.json()
    except Exception:
        data = None

    if 200 <= resp.status_code < 300:

        # Try to extract the assistant 'content' text from the common response
        # shape: data -> completions -> <model> -> completion -> choices[0] -> message -> content

        if isinstance(data, dict):
            completions = data.get('data', {}).get('completions', {})

            # If we passed a single model, try to use that key; otherwise take first
            model_key = None

            if models and len(models) == 1:
                model_key = models[0]
            if not model_key and isinstance(completions, dict) and len(completions) > 0:
                model_key = next(iter(completions.keys()))

            if model_key and model_key in completions:
                try:
                    content = completions[model_key]['completion']['choices'][0]['message']['content']
                    return content
                
                except Exception:
                    pass

        # Fallback: return raw text or the full JSON string
        if data is None:
            return resp.text
        
        try:
            return json.dumps(data, ensure_ascii=False)
        except Exception:
            return str(data)

    return {"error": f"status={resp.status_code}", "response": data if data is not None else resp.text}


### Configuración General

In [ ]:
def _import_prompts() -> Dict[str, str]:
    """Load prompt_1..prompt_3 from prompts.py and return as a dict.
    If prompts.py isn't present, returns an empty dict.
    """
    try:
        from Straico import prompts as p  # type: ignore
    except Exception:
        try:
            import prompts as p  # type: ignore
        except Exception:
            return {}

    out: Dict[str, str] = {}
    for name in ("prompt_1", "prompt_2", "prompt_3"):
        if hasattr(p, name):
            out[name] = getattr(p, name)
    return out

#### Modelos a usar

In [ ]:
# Cargar modelos desde el archivo text_model_ids.txt
MODELS_FILE = 'Straico/text_model_usefull.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

DEFAULT_MODELS = load_models_from_file(MODELS_FILE)

In [ ]:
DEFAULT_MODELS

#### Ejecusión

In [ ]:
DATASET_PATH = 'modismos_Dataset_Cleaned.csv'
RESPONSES_DIR = 'Straico'

# Calcular N_ROWS automáticamente
with open(DATASET_PATH, encoding='utf-8') as f:
    N_ROWS = sum(1 for _ in csv.DictReader(f, delimiter=';'))

print(f"Total de filas en el dataset: {N_ROWS}")

# Cargar prompts
PROMPTS = _import_prompts()
print(f"Prompts cargados: {list(PROMPTS.keys())}")

In [ ]:
def cargar_dataset(n_rows=None):
    """Carga el dataset y retorna una lista de diccionarios con modismo, significado y ejemplo."""
    rows = []
    seen_modismos = set()
    
    with open(DATASET_PATH, encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for r in reader:
            modismo = r.get('modismo', '').strip()
            if not modismo or modismo.casefold() in seen_modismos:
                continue
            seen_modismos.add(modismo.casefold())
            
            rows.append({
                'modismo': modismo,
                'significado': r.get('significado', '').strip(),
                'ejemplo': r.get('ejemplo', '').strip()
            })
            
            if n_rows and len(rows) >= n_rows:
                break
    
    return rows


def sanitize_model_name(model_name):
    """Convierte nombres de modelos en nombres válidos para nombres de archivos."""
    return model_name.replace('/', '_').replace(':', '_').replace('-', '_').replace('.', '_')


def save_json(filepath, data):
    """Guarda datos en un archivo JSON."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ Guardado: {filepath}")


def save_model_response(base_dir, prompt_name, model_name, data):
    """Guarda la respuesta de un modelo específico en su propia carpeta.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        model_name: Nombre del modelo
        data: Lista de respuestas del modelo
    """
    model_safe_name = sanitize_model_name(model_name)
    model_dir = os.path.join(base_dir, prompt_name, model_safe_name)
    filepath = os.path.join(model_dir, f"{model_safe_name}.json")
    save_json(filepath, data)


def save_consolidated_response(base_dir, prompt_name, all_models_data):
    """Guarda todas las respuestas consolidadas en un solo archivo.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        all_models_data: Diccionario con {model_name: [respuestas]}
    """
    filepath = os.path.join(base_dir, prompt_name, "all_models.json")
    save_json(filepath, all_models_data)


def save_partial_progress(base_dir, prompt_name, progress_data):
    """Guarda el progreso parcial durante la ejecución.
    
    Args:
        base_dir: Directorio base (ej: 'Straico')
        prompt_name: Nombre del prompt (ej: 'Prompt 1')
        progress_data: Datos parciales acumulados
    """
    filepath = os.path.join(base_dir, prompt_name, "progress.json")
    save_json(filepath, progress_data)

---
## PROMPT 1: Modismo → Definición

**Input**: `modismo`  
**Output**: `definicion`  
**Guardado**: 
- `Straico/Prompt 1/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 1/all_models.json` (consolidado)
- `Straico/Prompt 1/progress.json` (progreso parcial)

In [ ]:
def run_prompt_1(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 1: Dada la palabra/modismo → Generar definición
    INPUT: modismo
    OUTPUT: definicion
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 1: Modismo -> Definicion")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    print(f"Modelos a consultar: {len(models)}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_1')
    if not template:
        print("ERROR: prompt_1 no encontrado")
        return

    # Verificar progreso previo
    progress_path = os.path.join(RESPONSES_DIR, "Prompt 1", "progress.json")
    if os.path.exists(progress_path):
        with open(progress_path, 'r', encoding='utf-8') as f:
            progress_data = json.load(f)
        print("Progreso encontrado. Continuando desde el último punto...")
        all_models_data = progress_data.get("data", {})
        start_model_idx = progress_data.get("current_model_idx", 0)
        start_row_idx = progress_data.get("current_row_idx", 0)
    else:
        all_models_data = {model: [] for model in models}
        start_model_idx = 0
        start_row_idx = 0

    progress_data = {
        "current_model_idx": start_model_idx,
        "current_row_idx": start_row_idx,
        "total_rows": len(dataset),
        "models": models,
        "data": all_models_data
    }

    # Procesar modelo por modelo
    for model_idx, model in enumerate(models[start_model_idx:], start=start_model_idx):
        print(f"\nProcesando modelo {model_idx + 1}/{len(models)}: {model}")

        for row_idx, row in enumerate(tqdm(dataset[start_row_idx:], desc=f"Modismos para {model}", position=0), start=start_row_idx + 1):
            modismo = row['modismo']

            # Armar el prompt reemplazando placeholders
            prompt_text = template.replace('{{modismo}}', modismo)

            # Obtener respuesta del modelo
            resp = send_prompt(prompt_text, models=[model])

            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}

            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }

            all_models_data[model].append(entry)

            # Actualizar progreso
            progress_data["current_model_idx"] = model_idx
            progress_data["current_row_idx"] = row_idx
            progress_data["data"] = all_models_data

            # Guardar progreso parcial cada 100 filas
            if row_idx % 100 == 0:
                save_partial_progress(RESPONSES_DIR, "Prompt 1", progress_data)
                save_model_response(RESPONSES_DIR, "Prompt 1", model, all_models_data[model])

        # Guardar respuestas del modelo al finalizar
        save_model_response(RESPONSES_DIR, "Prompt 1", model, all_models_data[model])

        # Reiniciar el índice de filas al cambiar de modelo
        start_row_idx = 0

    # Guardar consolidado final
    print("\nGuardando archivo consolidado...")
    save_consolidated_response(RESPONSES_DIR, "Prompt 1", all_models_data)

    # Eliminar progreso al completar
    if os.path.exists(progress_path):
        os.remove(progress_path)

    print("\n" + "=" * 80)
    print("PROMPT 1 COMPLETADO")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 1
run_prompt_1()

---
## PROMPT 2: Modismo → Es Modismo (Sí/No)

**Input**: `modismo`  
**Output**: `es_modismo` (Sí/No)  
**Guardado**: 
- `Straico/Prompt 2/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 2/all_models.json` (consolidado)
- `Straico/Prompt 2/progress.json` (progreso parcial)

In [ ]:
def run_prompt_2(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 2: Dada la palabra/modismo → Determinar si es modismo (Sí/No)
    INPUT: modismo
    OUTPUT: es_modismo
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 2: Modismo -> Es Modismo (Si/No)")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    print(f"Dataset cargado: {len(dataset)} filas")
    print(f"Modelos a consultar: {len(models)}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_2')
    if not template:
        print("ERROR: prompt_2 no encontrado")
        return

    # Verificar progreso previo
    progress_path = os.path.join(RESPONSES_DIR, "Prompt 2", "progress.json")
    if os.path.exists(progress_path):
        with open(progress_path, 'r', encoding='utf-8') as f:
            progress_data = json.load(f)
        print("Progreso encontrado. Continuando desde el último punto...")
        all_models_data = progress_data.get("data", {})
        start_model_idx = progress_data.get("current_model_idx", 0)
        start_row_idx = progress_data.get("current_row_idx", 0)
    else:
        all_models_data = {model: [] for model in models}
        start_model_idx = 0
        start_row_idx = 0

    progress_data = {
        "current_model_idx": start_model_idx,
        "current_row_idx": start_row_idx,
        "total_rows": len(dataset),
        "models": models,
        "data": all_models_data
    }

    # Procesar modelo por modelo
    for model_idx, model in enumerate(models[start_model_idx:], start=start_model_idx):
        print(f"\nProcesando modelo {model_idx + 1}/{len(models)}: {model}")

        for row_idx, row in enumerate(tqdm(dataset[start_row_idx:], desc=f"Modismos para {model}", position=0), start=start_row_idx + 1):
            modismo = row['modismo']

            # Armar el prompt reemplazando placeholders
            prompt_text = template.replace('{{modismo}}', modismo)

            # Obtener respuesta del modelo
            resp = send_prompt(prompt_text, models=[model])

            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}

            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "model": model,
                "response": response_data
            }

            all_models_data[model].append(entry)

            # Actualizar progreso
            progress_data["current_model_idx"] = model_idx
            progress_data["current_row_idx"] = row_idx
            progress_data["data"] = all_models_data

            # Guardar progreso parcial cada 100 filas
            if row_idx % 100 == 0:
                save_partial_progress(RESPONSES_DIR, "Prompt 2", progress_data)
                save_model_response(RESPONSES_DIR, "Prompt 2", model, all_models_data[model])

        # Guardar respuestas del modelo al finalizar
        save_model_response(RESPONSES_DIR, "Prompt 2", model, all_models_data[model])

        # Reiniciar el índice de filas al cambiar de modelo
        start_row_idx = 0

    # Guardar consolidado final
    print("\nGuardando archivo consolidado...")
    save_consolidated_response(RESPONSES_DIR, "Prompt 2", all_models_data)

    # Eliminar progreso al completar
    if os.path.exists(progress_path):
        os.remove(progress_path)

    print("\n" + "=" * 80)
    print("PROMPT 2 COMPLETADO")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 2
run_prompt_2()

---
## PROMPT 3: Modismo + Ejemplo → Literal + Definición

**Input**: `modismo` + `ejemplo`  
**Output**: `literal` + `definicion`  
**Guardado**: 
- `Straico/Prompt 3/{modelo}/{modelo}.json` (por cada modelo)
- `Straico/Prompt 3/all_models.json` (consolidado)
- `Straico/Prompt 3/progress.json` (progreso parcial)

In [ ]:
def run_prompt_3(models=DEFAULT_MODELS, n_rows=N_ROWS):
    """
    PROMPT 3: Dado modismo + ejemplo → Generar literal + definición
    INPUT: modismo + ejemplo
    OUTPUT: literal + definicion
    """
    print("=" * 80)
    print("EJECUTANDO PROMPT 3: Modismo + Ejemplo -> Literal + Definicion")
    print("=" * 80)

    # Cargar dataset
    dataset = cargar_dataset(n_rows)
    dataset_with_examples = [row for row in dataset if row['ejemplo']]
    print(f"Dataset cargado: {len(dataset_with_examples)} filas con ejemplo")
    print(f"Modelos a consultar: {len(models)}")

    # Obtener template del prompt
    template = PROMPTS.get('prompt_3')
    if not template:
        print("ERROR: prompt_3 no encontrado")
        return

    # Verificar progreso previo
    progress_path = os.path.join(RESPONSES_DIR, "Prompt 3", "progress.json")
    if os.path.exists(progress_path):
        with open(progress_path, 'r', encoding='utf-8') as f:
            progress_data = json.load(f)
        print("Progreso encontrado. Continuando desde el último punto...")
        all_models_data = progress_data.get("data", {})
        start_model_idx = progress_data.get("current_model_idx", 0)
        start_row_idx = progress_data.get("current_row_idx", 0)
    else:
        all_models_data = {model: [] for model in models}
        start_model_idx = 0
        start_row_idx = 0

    progress_data = {
        "current_model_idx": start_model_idx,
        "current_row_idx": start_row_idx,
        "total_rows": len(dataset_with_examples),
        "models": models,
        "data": all_models_data
    }

    # Procesar modelo por modelo
    for model_idx, model in enumerate(models[start_model_idx:], start=start_model_idx):
        print(f"\nProcesando modelo {model_idx + 1}/{len(models)}: {model}")

        for row_idx, row in enumerate(tqdm(dataset_with_examples[start_row_idx:], desc=f"Modismos para {model}", position=0), start=start_row_idx + 1):
            modismo = row['modismo']
            ejemplo = row['ejemplo']

            # Armar el prompt reemplazando placeholders
            prompt_text = template.replace('{{modismo}}', modismo).replace('{{ejemplo}}', ejemplo)

            # Obtener respuesta del modelo
            resp = send_prompt(prompt_text, models=[model])

            # Procesar respuesta
            if isinstance(resp, str):
                try:
                    parsed = json.loads(resp)
                    response_data = parsed
                except:
                    response_data = {"raw_response": resp}
            elif isinstance(resp, dict):
                response_data = resp
            else:
                response_data = {"raw_response": str(resp)}

            # Agregar metadatos
            entry = {
                "modismo": modismo,
                "ejemplo": ejemplo,
                "model": model,
                "response": response_data
            }

            all_models_data[model].append(entry)

            # Actualizar progreso
            progress_data["current_model_idx"] = model_idx
            progress_data["current_row_idx"] = row_idx
            progress_data["data"] = all_models_data

            # Guardar progreso parcial cada 100 filas
            if row_idx % 100 == 0:
                save_partial_progress(RESPONSES_DIR, "Prompt 3", progress_data)
                save_model_response(RESPONSES_DIR, "Prompt 3", model, all_models_data[model])

        # Guardar respuestas del modelo al finalizar
        save_model_response(RESPONSES_DIR, "Prompt 3", model, all_models_data[model])

        # Reiniciar el índice de filas al cambiar de modelo
        start_row_idx = 0

    # Guardar consolidado final
    print("\nGuardando archivo consolidado...")
    save_consolidated_response(RESPONSES_DIR, "Prompt 3", all_models_data)

    # Eliminar progreso al completar
    if os.path.exists(progress_path):
        os.remove(progress_path)

    print("\n" + "=" * 80)
    print("PROMPT 3 COMPLETADO")
    print("=" * 80)

In [ ]:
# Ejecutar Prompt 3
run_prompt_3()